In [ ]:
import os
import sys
sys.path.append(os.pardir)

import numpy as np
import torch as th
import torch.distributed as dist

from guided_diffusion import dist_util, logger
from guided_diffusion.image_datasets import load_data_sde, ImageDataset
from guided_diffusion.script_util import (
    NUM_CLASSES,
    model_and_diffusion_defaults,
    create_model_and_diffusion,
    add_dict_to_argparser,
    args_to_dict,
)
from torchvision import utils
from easydict import EasyDict
import PIL.Image as Image
import torchvision
from utils import *

to_pil = torchvision.transforms.ToPILImage()
device = 'cuda:3'
batch_size = 4

In [ ]:
args_src = EasyDict({'attention_resolutions':'16', 
        'batch_size':batch_size, 
        'channel_mult':'', 
        'class_cond':False, 
        'clip_denoised':True, 
        'diffusion_steps':1000, 
        'dropout':0.0, 
        'image_size':256, 
        'learn_sigma':True, 
        'model_path':'../logs/aahq_limited_transfer/model080000.pt', 
        'noise_schedule':'linear', 
        'num_channels':128, 
        'num_head_channels':64, 
        'num_heads':4, 
        'num_heads_upsample':-1, 
        'num_res_blocks':1, 
        'num_samples':10000, 
        'p2_gamma':0, 
        'p2_k':1, 
        'predict_xstart':False, 
        'resblock_updown':True, 
        'rescale_learned_sigmas':False,
        'rescale_timesteps':False, 
        'sample_dir':'',
        'timestep_respacing':'ddim40', 
        'use_checkpoint':False,
        'use_ddim':True, 
        'use_fp16':True, 
        'use_kl':False, 
        'use_new_attention_order':False, 
        'use_scale_shift_norm':True})
model_src, diffusion = create_model_and_diffusion(
    **args_to_dict(args_src, model_and_diffusion_defaults().keys())
)

model_src.load_state_dict(
    dist_util.load_state_dict(args_src.model_path, map_location="cpu")
)
model_src.to(device)
model_src.convert_to_fp16()
model_src.eval()
print('done')

In [ ]:
exp_name = 'aahq_limited_distill_p2_0.05_5_aux_0.3_50'
# epochs = ['010000','020000','030000','040000']
epochs = ['005000','010000','015000','020000','030000','040000']

args_trg_0 = EasyDict({'attention_resolutions':'16', 
        'batch_size':batch_size, 
        'channel_mult':'', 
        'class_cond':False, 
        'clip_denoised':True, 
        'diffusion_steps':1000, 
        'dropout':0.0, 
        'image_size':256, 
        'learn_sigma':True, 
        'model_path':f'../logs/{exp_name}/model{epochs[0]}.pt', 
        'noise_schedule':'linear', 
        'num_channels':128, 
        'num_head_channels':64, 
        'num_heads':4, 
        'num_heads_upsample':-1, 
        'num_res_blocks':1, 
        'num_samples':10000, 
        'p2_gamma':0, 
        'p2_k':1, 
        'predict_xstart':False, 
        'resblock_updown':True, 
        'rescale_learned_sigmas':False,
        'rescale_timesteps':False, 
        'sample_dir':'',
        'timestep_respacing':'ddim40', 
        'use_checkpoint':False,
        'use_ddim':True, 
        'use_fp16':True, 
        'use_kl':False, 
        'use_new_attention_order':False, 
        'use_scale_shift_norm':True})
model_trg_0, diffusion = create_model_and_diffusion(
    **args_to_dict(args_trg_0, model_and_diffusion_defaults().keys())
)

model_trg_0.load_state_dict(
    dist_util.load_state_dict(args_trg_0.model_path, map_location="cpu")
)
model_trg_0.to(device)
model_trg_0.convert_to_fp16()
model_trg_0.eval()
print('done')

args_trg_1 = EasyDict({'attention_resolutions':'16', 
        'batch_size':batch_size, 
        'channel_mult':'', 
        'class_cond':False, 
        'clip_denoised':True, 
        'diffusion_steps':1000, 
        'dropout':0.0, 
        'image_size':256, 
        'learn_sigma':True, 
        'model_path':f'../logs/{exp_name}/model{epochs[1]}.pt', 
        'noise_schedule':'linear', 
        'num_channels':128, 
        'num_head_channels':64, 
        'num_heads':4, 
        'num_heads_upsample':-1, 
        'num_res_blocks':1, 
        'num_samples':10000, 
        'p2_gamma':0, 
        'p2_k':1, 
        'predict_xstart':False, 
        'resblock_updown':True, 
        'rescale_learned_sigmas':False,
        'rescale_timesteps':False, 
        'sample_dir':'',
        'timestep_respacing':'ddim40', 
        'use_checkpoint':False,
        'use_ddim':True, 
        'use_fp16':True, 
        'use_kl':False, 
        'use_new_attention_order':False, 
        'use_scale_shift_norm':True})
model_trg_1, diffusion = create_model_and_diffusion(
    **args_to_dict(args_trg_1, model_and_diffusion_defaults().keys())
)

model_trg_1.load_state_dict(
    dist_util.load_state_dict(args_trg_1.model_path, map_location="cpu")
)
model_trg_1.to(device)
model_trg_1.convert_to_fp16()
model_trg_1.eval()
print('done')

args_trg_2 = EasyDict({'attention_resolutions':'16', 
        'batch_size':batch_size, 
        'channel_mult':'', 
        'class_cond':False, 
        'clip_denoised':True, 
        'diffusion_steps':1000, 
        'dropout':0.0, 
        'image_size':256, 
        'learn_sigma':True, 
        'model_path':f'../logs/{exp_name}/model{epochs[2]}.pt', 
        'noise_schedule':'linear', 
        'num_channels':128, 
        'num_head_channels':64, 
        'num_heads':4, 
        'num_heads_upsample':-1, 
        'num_res_blocks':1, 
        'num_samples':10000, 
        'p2_gamma':0, 
        'p2_k':1, 
        'predict_xstart':False, 
        'resblock_updown':True, 
        'rescale_learned_sigmas':False,
        'rescale_timesteps':False, 
        'sample_dir':'',
        'timestep_respacing':'ddim40', 
        'use_checkpoint':False,
        'use_ddim':True, 
        'use_fp16':True, 
        'use_kl':False, 
        'use_new_attention_order':False, 
        'use_scale_shift_norm':True})
model_trg_2, diffusion = create_model_and_diffusion(
    **args_to_dict(args_trg_2, model_and_diffusion_defaults().keys())
)

model_trg_2.load_state_dict(
    dist_util.load_state_dict(args_trg_2.model_path, map_location="cpu")
)
model_trg_2.to(device)
model_trg_2.convert_to_fp16()
model_trg_2.eval()
print('done')
args_trg_3 = EasyDict({'attention_resolutions':'16', 
        'batch_size':batch_size, 
        'channel_mult':'', 
        'class_cond':False, 
        'clip_denoised':True, 
        'diffusion_steps':1000, 
        'dropout':0.0, 
        'image_size':256, 
        'learn_sigma':True, 
        'model_path':f'../logs/{exp_name}/model{epochs[3]}.pt', 
        'noise_schedule':'linear', 
        'num_channels':128, 
        'num_head_channels':64, 
        'num_heads':4, 
        'num_heads_upsample':-1, 
        'num_res_blocks':1, 
        'num_samples':10000, 
        'p2_gamma':0, 
        'p2_k':1, 
        'predict_xstart':False, 
        'resblock_updown':True, 
        'rescale_learned_sigmas':False,
        'rescale_timesteps':False, 
        'sample_dir':'',
        'timestep_respacing':'ddim40', 
        'use_checkpoint':False,
        'use_ddim':True, 
        'use_fp16':True, 
        'use_kl':False, 
        'use_new_attention_order':False, 
        'use_scale_shift_norm':True})
model_trg_3, diffusion = create_model_and_diffusion(
    **args_to_dict(args_trg_3, model_and_diffusion_defaults().keys())
)

model_trg_3.load_state_dict(
    dist_util.load_state_dict(args_trg_3.model_path, map_location="cpu")
)
model_trg_3.to(device)
model_trg_3.convert_to_fp16()
model_trg_3.eval()
print('done')
args_trg_4 = EasyDict({'attention_resolutions':'16', 
        'batch_size':batch_size, 
        'channel_mult':'', 
        'class_cond':False, 
        'clip_denoised':True, 
        'diffusion_steps':1000, 
        'dropout':0.0, 
        'image_size':256, 
        'learn_sigma':True, 
        'model_path':f'../logs/{exp_name}/model{epochs[4]}.pt', 
        'noise_schedule':'linear', 
        'num_channels':128, 
        'num_head_channels':64, 
        'num_heads':4, 
        'num_heads_upsample':-1, 
        'num_res_blocks':1, 
        'num_samples':10000, 
        'p2_gamma':0, 
        'p2_k':1, 
        'predict_xstart':False, 
        'resblock_updown':True, 
        'rescale_learned_sigmas':False,
        'rescale_timesteps':False, 
        'sample_dir':'',
        'timestep_respacing':'ddim40', 
        'use_checkpoint':False,
        'use_ddim':True, 
        'use_fp16':True, 
        'use_kl':False, 
        'use_new_attention_order':False, 
        'use_scale_shift_norm':True})
model_trg_4, diffusion = create_model_and_diffusion(
    **args_to_dict(args_trg_4, model_and_diffusion_defaults().keys())
)

model_trg_4.load_state_dict(
    dist_util.load_state_dict(args_trg_4.model_path, map_location="cpu")
)
model_trg_4.to(device)
model_trg_4.convert_to_fp16()
model_trg_4.eval()
print('done')

args_trg_5 = EasyDict({'attention_resolutions':'16', 
        'batch_size':batch_size, 
        'channel_mult':'', 
        'class_cond':False, 
        'clip_denoised':True, 
        'diffusion_steps':1000, 
        'dropout':0.0, 
        'image_size':256, 
        'learn_sigma':True, 
        'model_path':f'../logs/{exp_name}/model{epochs[5]}.pt', 
        'noise_schedule':'linear', 
        'num_channels':128, 
        'num_head_channels':64, 
        'num_heads':4, 
        'num_heads_upsample':-1, 
        'num_res_blocks':1, 
        'num_samples':10000, 
        'p2_gamma':0, 
        'p2_k':1, 
        'predict_xstart':False, 
        'resblock_updown':True, 
        'rescale_learned_sigmas':False,
        'rescale_timesteps':False, 
        'sample_dir':'',
        'timestep_respacing':'ddim40', 
        'use_checkpoint':False,
        'use_ddim':True, 
        'use_fp16':True, 
        'use_kl':False, 
        'use_new_attention_order':False, 
        'use_scale_shift_norm':True})
model_trg_5, diffusion = create_model_and_diffusion(
    **args_to_dict(args_trg_5, model_and_diffusion_defaults().keys())
)

model_trg_5.load_state_dict(
    dist_util.load_state_dict(args_trg_5.model_path, map_location="cpu")
)
model_trg_5.to(device)
model_trg_5.convert_to_fp16()
model_trg_5.eval()
print('done')

In [ ]:
data = load_data_sde(
        # data_dir='/mnt/raid/metface/256/',
        data_dir='../FFHQ_partial/',
        batch_size=args_src.batch_size,
        image_size=args_src.image_size,
        class_cond=args_src.class_cond,
        deterministic=True
    )

In [ ]:
t_end = 20
sample_fn = diffusion.ddim_sample_loop
model_kwargs = {}
save_idx = 0
num=120
# while save_idx<10000:
for idx, batch in enumerate(data):
    if idx < num:
        continue
    if idx > num+10:
        break
    img, cond = batch
    img = img.to(device)
    img_pil = ((img + 1) * 127.5).clamp(0, 255).to(th.uint8) 
    noise = th.randn_like(img)
    t = th.tensor([t_end] * img.shape[0], device=device)
    perturbed = diffusion.q_sample(img, t, noise=noise)
    # perturbed = noise
    
    sample_src = sample_fn(
        model_src,
        (args_src.batch_size, 3, args_src.image_size, args_src.image_size),
        clip_denoised=args_src.clip_denoised,
        model_kwargs=model_kwargs,
        noise=perturbed,
        t_start=0,
        t_end=t_end
    )
    sample_src = ((sample_src + 1) * 127.5).clamp(0, 255).to(th.uint8)
    
    sample_trg_0 = sample_fn(
        model_trg_0,
        (args_src.batch_size, 3, args_src.image_size, args_src.image_size),
        clip_denoised=args_src.clip_denoised,
        model_kwargs=model_kwargs,
        noise=perturbed,
        t_start=0,
        t_end=t_end
    )
    sample_trg_0 = ((sample_trg_0 + 1) * 127.5).clamp(0, 255).to(th.uint8)    
    sample_trg_1 = sample_fn(
        model_trg_1,
        (args_src.batch_size, 3, args_src.image_size, args_src.image_size),
        clip_denoised=args_src.clip_denoised,
        model_kwargs=model_kwargs,
        noise=perturbed,
        t_start=0,
        t_end=t_end
    )
    sample_trg_1 = ((sample_trg_1 + 1) * 127.5).clamp(0, 255).to(th.uint8)
    sample_trg_2 = sample_fn(
        model_trg_2,
        (args_src.batch_size, 3, args_src.image_size, args_src.image_size),
        clip_denoised=args_src.clip_denoised,
        model_kwargs=model_kwargs,
        noise=perturbed,
        t_start=0,
        t_end=t_end
    )
    sample_trg_2 = ((sample_trg_2 + 1) * 127.5).clamp(0, 255).to(th.uint8)
    sample_trg_2 = sample_fn(
        model_trg_2,
        (args_src.batch_size, 3, args_src.image_size, args_src.image_size),
        clip_denoised=args_src.clip_denoised,
        model_kwargs=model_kwargs,
        noise=perturbed,
        t_start=0,
        t_end=t_end
    )
    sample_trg_2 = ((sample_trg_2 + 1) * 127.5).clamp(0, 255).to(th.uint8)
    sample_trg_3 = sample_fn(
        model_trg_3,
        (args_src.batch_size, 3, args_src.image_size, args_src.image_size),
        clip_denoised=args_src.clip_denoised,
        model_kwargs=model_kwargs,
        noise=perturbed,
        t_start=0,
        t_end=t_end
    )
    sample_trg_3 = ((sample_trg_3 + 1) * 127.5).clamp(0, 255).to(th.uint8)
    sample_trg_4 = sample_fn(
        model_trg_4,
        (args_src.batch_size, 3, args_src.image_size, args_src.image_size),
        clip_denoised=args_src.clip_denoised,
        model_kwargs=model_kwargs,
        noise=perturbed,
        t_start=0,
        t_end=t_end
    )
    sample_trg_4 = ((sample_trg_4 + 1) * 127.5).clamp(0, 255).to(th.uint8)
    sample_trg_5 = sample_fn(
        model_trg_5,
        (args_src.batch_size, 3, args_src.image_size, args_src.image_size),
        clip_denoised=args_src.clip_denoised,
        model_kwargs=model_kwargs,
        noise=perturbed,
        t_start=0,
        t_end=t_end
    )
    sample_trg_5 = ((sample_trg_5 + 1) * 127.5).clamp(0, 255).to(th.uint8)
    
    
    display(concat_eight_batch(batch_size, img_pil, sample_src, sample_trg_0, sample_trg_1, sample_trg_2, sample_trg_3, sample_trg_4, sample_trg_5))
    
    # for i in range(img.shape[0]):
    #     out_path = f'/home/jiwan.hur/samples/sdeit_metface2ffhq_30_0/{save_idx}.png'
    #     torchvision.utils.save_image(
    #         sample_src[i].unsqueeze(0),
    #         out_path,
    #         nrow=1,
    #         normalize=True,
    #         range=(-1, 1),
    #     )
    #     save_idx += 1
    
    # if save_idx > 10000:
    #     break
        